### Experiment 1
* Tokenizer - Keras
* Embeddings training from uniform; separate for each one
* ** Optimizers and learning rate - test to check local cv vs lb **
* Extra features - None for now

In [1]:
import copy
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import sys
import os
import re
import threading
import multiprocessing

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import lightgbm as lgb
from sklearn import metrics
import gc

# from __future__ import print_function
np.random.seed(786)  # for reproducibility

from keras.models import Sequential, Model, load_model
from keras.layers import *
from keras.optimizers import *
from keras.utils import np_utils
from keras.layers.convolutional import Convolution1D, MaxPooling1D, ZeroPadding1D, AveragePooling1D
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.layers.wrappers import TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.callbacks import Callback, ModelCheckpoint
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier,  KerasRegressor
#Some classes
#Functions we need - Feature Selector, Fasttext_Estimator, Preprocessing Transformer, Binary_Encoder
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from pandas.api.types import is_numeric_dtype, is_string_dtype
from scipy.sparse.csr import csr_matrix
from sklearn.metrics import mean_squared_error, make_scorer

def rmse(y_true, y_pred):
    print(np.min(y_pred), np.max(y_pred))
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_sklearn = make_scorer(rmse, greater_is_better=False)

def get_obj_cols(df):
    """Return columns with object dtypes"""
    obj_cols = []
    for idx, dt in enumerate(df.dtypes):
        if dt == 'object':
            obj_cols.append(df.columns.values[idx])

    return obj_cols


def convert_input(X):
    """if input not a dataframe convert it to one"""
    if not isinstance(X, pd.DataFrame):
        if isinstance(X, list):
            X = pd.DataFrame(np.array(X))
        elif isinstance(X, (np.generic, np.ndarray)):
            X = pd.DataFrame(X)
        elif isinstance(X, csr_matrix):
            X = pd.SparseDataFrame(X)
        else:
            raise ValueError('Unexpected input type: %s' % (str(type(X))))

        #X = X.apply(lambda x: pd.to_numeric(x, errors='ignore'))
    return X

class FeatureSelector(BaseEstimator, TransformerMixin):
    """ Class to do subset of features in sklearn pipeline"""
    def __init__(self, cols=None, return_df=True, verbose=0):
        self.cols = cols
        self.return_df = return_df
        self.verbose = verbose
        
    def fit(self, X, y=None):
        #Do nothing
        return self
    
    def transform(self, X, y=None):
        #if the input dataset isn't already a dataframe, convert it to one
        X = X.copy(deep=True)
        X = convert_input(X)
        X = X.loc[:, self.col]
        
        if self.verbose:
            print("Selecting columns are {}".format(self.col))
        if self.return_df:
            return X
        else:
            return X.values
    
      
#Data reading function
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.)

def read_data(in_path, out_path):
    if False and os.path.exists(os.path.join(out_path, 'train_2.pkl')) and os.path.exists(os.path.join(out_path, 'test_2.pkl')):
        train_data = pd.read_pickle(os.path.join(out_path, 'train_2.pkl'))
        test_data  = pd.read_pickle(os.path.join(out_path, 'test_2.pkl'))
        
        return train_data, test_data
    
    else:
        train_data = pd.read_table(os.path.join(in_path, 'train.tsv'))
        test_data  = pd.read_table(os.path.join(in_path, 'test.tsv'))
    
        train_rows = len(train_data)
        data = pd.concat([train_data, test_data], ignore_index=True)
    
        data['cat1'] = data['category_name'].apply(lambda x: str(x).split('/')[0])
        data['cat2'] = data['category_name'].apply(lambda x: str(x).split('/')[1] 
                                                   if len(str(x).split('/')) > 1 else -1)
        data['cat3'] = data['category_name'].apply(lambda x: ' '.join(str(x).split('/')[2:]) 
                                                   if len(str(x).split('/')) > 2 else -1)
        data.fillna(-1, inplace=True)
        cat_cols = ['category_name', 'brand_name', 'cat1', 'cat2', 'cat3', 'item_condition_id']
        for col in cat_cols:
            data[col] = LabelEncoder().fit_transform(data[col].astype(str))
        
        #tkn_desc = Tokenizer(50000)
        #tkn_desc.fit_on_texts(data.item_description.astype(str))
        #data['desc_seq'] = pad_sequences(tkn_desc.texts_to_sequences(data.item_description.astype(str)),
        #                                 maxlen=100, padding='post', truncating='post')
        
        #tkn_name = Tokenizer(4000)
        #tkn_name.fit_on_texts(data.name.astype(str))
        #data['name_seq'] = pad_sequences(tkn_name.texts_to_sequences(data.name.astype(str)),
        #                                 maxlen=6, padding='post', truncating='post')
        
        
        train_data = data.loc[: train_rows - 1, :].reset_index(drop=True)
        train_data = train_data.loc[(train_data.price >= 1) & (train_data.price <= 2000), :].reset_index(drop=True)
        test_data  = data.loc[train_rows: , :].reset_index(drop=True)
        
        del train_data['test_id']
        del test_data['train_id']
        del data 
        test_data['test_id'] = test_data['test_id'].astype(int)
        #train_data.to_pickle(os.path.join(out_path, 'train_2.pkl'))
        #test_data.to_pickle(os.path.join(out_path, 'test_2.pkl'))
        
        return train_data, test_data
        


/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/mohsin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [22]:
class EM_NNRegressor(BaseEstimator, RegressorMixin):
    
    def __init__(self, embed_cols=None, dense_cols=None, embed_dims=None, 
                 text_embed_cols=None, text_embed_seq_lens=None, 
                 text_embed_dims=None, text_embed_tokenizers=None,
                 num_layers=2, multiprocess=False,
                layer_activations=None, layer_dims=None,layer_dropouts=None, epochs=20, batchsize=32,
                optimizer_kwargs=None, val_size=0.1, verbose=1, seed=1):
        
        self.embed_cols = embed_cols
        self.dense_cols = dense_cols
        self.embed_dims = embed_dims
        self.text_embed_cols = text_embed_cols
        self.text_embed_dims = text_embed_dims
        self.text_embed_tokenizers = text_embed_tokenizers
        self.text_embed_seq_lens = text_embed_seq_lens
        self.dense_dims = None
        self.num_layers = num_layers
        self.layer_dims = layer_dims
        self.layer_activations = layer_activations
        self.layer_dropouts = layer_dropouts
        self.epochs = epochs
        self.batchsize = batchsize
        self.optimizer_kwargs = optimizer_kwargs
        self.val_size = val_size
        self.verbose = verbose
        self.multiprocess = multiprocess
        self.seed = seed
        self.model = None
        if self.dense_cols:
            self.dense_dims = len(self.dense_cols)
            
    def _splitX(self, X):
        X_splits = []
        
        if self.embed_cols:
            for col in self.embed_cols :
                X_splits.append(X[col].values.reshape(X.shape[0], -1))
                
        if self.text_embed_cols:
            for i, (col, tok) in enumerate(zip(self.text_embed_cols, self.text_embed_tokenizers)):
                max_features = self.text_embed_dims[i][0]
                max_len = self.text_embed_seq_lens[i]
                input_text = X[col].astype(str)
                x_train = tok.texts_to_sequences(input_text)
                print(np.mean([len(l) for l in x_train]))
                x_train = sequence.pad_sequences(x_train, maxlen=max_len)
                X_splits.append(np.array(x_train).reshape(X.shape[0], -1))
                
        if self.dense_cols:
            X_splits.append(X[self.dense_cols].values.reshape(X.shape[0], -1))
            
        return X_splits
    
    
    def _build_model(self):
        model_inputs = []
        model_layers = []
        
        if self.embed_cols:
            for col, dim in zip(self.embed_cols, self.embed_dims):
                x1 = Input( shape=(1,), name=col)
                model_inputs.append(x1)
                x1 = Embedding(input_dim=dim[0], output_dim=dim[1], )(x1)
                #x1 = Dropout(0.1)(x1)
                x1 = Reshape(target_shape=(dim[1],))(x1)
                model_layers.append(x1)
                
        if self.text_embed_cols:
            for col, dim, seq_len in zip(self.text_embed_cols, 
                                                self.text_embed_dims, 
                                                self.text_embed_seq_lens):
                x3 = Input( shape=(seq_len,))
                model_inputs.append(x3)
                x3 = Embedding(input_dim=dim[0], output_dim=dim[1], input_length=seq_len)(x3)
                x3 = GlobalAveragePooling1D()(x3)
                x3 = Reshape(target_shape=(dim[1],))(x3)
                model_layers.append(x3)
                
        if self.dense_cols:
            x2 = Input( shape=(self.dense_dims, ), name='dense_cols')
            model_inputs.append(x2)
            model_layers.append(x2)
        print(model_layers)
        x = concatenate(model_layers)
        
        if self.num_layers > 0:
            for dim, drops in zip(self.layer_dims, self.layer_dropouts):
                x = BatchNormalization()(x)
                x = Dropout(rate=drops)(x)
                x = Dense(dim, kernel_initializer='he_normal')(x)
                x = LeakyReLU()(x)
        
        x = BatchNormalization()(x)
        x = Dropout(0.05)(x)
        output = Dense(1, activation='linear', kernel_initializer='normal')(x)
        
        model = Model(inputs=model_inputs, outputs=output)
        #print(model.summary())
        adam = Nadam(lr=0.0012, schedule_decay=0.01)
        #adam = Adam(lr=0.003, decay=0.003)
        #adam = SGD(lr=0.01, nesterov=True, momentum=0.9, decay=0.003)
        #adam = RMSprop(lr=0.004, decay=0.004)
        model.compile(optimizer=adam, loss='mean_squared_error' )
        
        return model 
    
    
    def fit(self, X, y):
        self.model = self._build_model()
        if self.val_size > 0:
            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.val_size, random_state=self.seed)
            print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
            
            callbacks= [ModelCheckpoint("embed_NN_"+str(self.seed)+".check", save_best_only=True, verbose=1)]
            if self.multiprocess == False:
                self.model.fit(self._splitX(X_train), y_train, batch_size=self.batchsize, epochs=self.epochs,
                               verbose=self.verbose,
                              validation_data=(self._splitX(X_val), y_val), shuffle=True,
                              callbacks=callbacks)
            else:
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=self.val_size, random_state=1)

        else:
            self.model.fit(self._splitX(X), y, batch_size=self.batchsize, epochs=self.epochs,
               verbose=self.verbose, shuffle=True)

        
        return self
    
    def predict(self, X, y=None):
        
        if self.model:
            model = load_model("embed_NN_"+str(self.seed)+".check")
            y_hat = model.predict(self._splitX(X))
        else:
            raise ValueError("Model not fit yet")
            
        return y_hat
        
def add_ngrams(text, ngram=2):
    word_list = str(text).lower().split(' ')
    out_list = [''.join(word_list[i:i+ngram]) for i in range(len(word_list))]
    return ' '.join(out_list[:-1])
        



In [3]:
#Read data
train_data, test_data = read_data("../input", "./")
print(train_data.shape, test_data.shape)
train_data.head()

(1481658, 11) (693359, 11)


,brand_name,category_name,item_condition_id,item_description,name,price,shipping,train_id,cat1,cat2,cat3
0,2,830,2,No description yet,MLB Cincinnati Reds T Shirt Size XL,10.0,1,0.0,5,103,774
1,3890,87,2,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,52.0,0,1.0,1,31,216
2,4589,1278,0,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,10.0,1,2.0,9,104,98
3,2,504,0,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,35.0,1,3.0,3,56,411
4,2,1205,0,Complete with certificate of authenticity,24K GOLD plated rose,44.0,0,4.0,9,59,543


In [4]:
train_data['item_desc2gram'] = train_data.item_description.apply(lambda x: add_ngrams(x, 2))
test_data['item_desc2gram'] = test_data.item_description.apply(lambda x: add_ngrams(x, 2))

In [5]:
print(train_data.shape, test_data.shape)
train_data.head()

(1481658, 12) (693359, 12)


,brand_name,category_name,item_condition_id,item_description,name,price,shipping,train_id,cat1,cat2,cat3,item_desc2gram
0,2,830,2,No description yet,MLB Cincinnati Reds T Shirt Size XL,10.0,1,0.0,5,103,774,nodescription descriptionyet
1,3890,87,2,This keyboard is in great condition and works ...,Razer BlackWidow Chroma Keyboard,52.0,0,1.0,1,31,216,thiskeyboard keyboardis isin ingreat greatcond...
2,4589,1278,0,Adorable top with a hint of lace and a key hol...,AVA-VIV Blouse,10.0,1,2.0,9,104,98,adorabletop topwith witha ahint hintof oflace ...
3,2,504,0,New with tags. Leather horses. Retail for [rm]...,Leather Horse Statues,35.0,1,3.0,3,56,411,newwith withtags. tags.leather leatherhorses. ...
4,2,1205,0,Complete with certificate of authenticity,24K GOLD plated rose,44.0,0,4.0,9,59,543,completewith withcertificate certificateof ofa...


In [6]:
print("Tokenizing data")
tok_name  = Tokenizer(20000)
tok_name.fit_on_texts(pd.concat([train_data, test_data])['name'].astype(str))

tok_desc= Tokenizer(50000)
tok_desc.fit_on_texts(pd.concat([train_data, test_data])['item_description'].astype(str))

tok_desc2 = Tokenizer(20000)
tok_desc2.fit_on_texts(pd.concat([train_data, test_data])['item_desc2gram'].astype(str))

Tokenizing data


In [7]:
X = train_data
y = np.log1p(train_data.price)

cvlist= list(KFold(5, random_state=786).split(X, y))

In [23]:
nnet1 = EM_NNRegressor(embed_cols=['brand_name','category_name','item_condition_id', 'cat1', 'cat2', 'cat3'], 
                  embed_dims=[(6000, 40),(1500, 30), (5,4), (15,4), (120, 10), (900, 20)],
                  text_embed_cols=['name', 'item_description', 'item_desc2gram'],
                  text_embed_dims=[(20000, 50), (50000, 50), (20000, 50)],
                  text_embed_seq_lens =[7, 60, 30],
                  text_embed_tokenizers = [tok_name, tok_desc, tok_desc2],
                  dense_cols=['shipping'],
                  epochs=5,
                  batchsize=2048 ,
                  num_layers = 1,
                  layer_dropouts=[0.2],
                  layer_dims=[200],
                  seed=1,
                  val_size=0.02,
                 )

oof_preds1 = cross_val_predict(nnet1, X, y, verbose=10, cv=cvlist)
score = rmse(y, oof_preds1)
print(score)


[<tf.Tensor 'reshape_136/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_137/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_138/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_139/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_140/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_141/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_142/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_143/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_144/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_27:0' shape=(?, 1) dtype=float32>]
(1161619, 12) (23707, 12) (1161619,) (23707,)
4.334399661162567
25.62508447261968
19.236752325848666
4.335765807567385
25.815370987472054
19.345889399755347
Train on 1161619 samples, validate on 23707 samples
Epoch 1/5
1161619/1161619 [==============================] - 17s 15us/step - loss: 1.0989 - val_loss: 0.2687
Epoch 2/5
1161619/1161619 [=============

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.4min remaining:    0.0s


[<tf.Tensor 'reshape_145/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_146/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_147/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_148/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_149/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_150/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_151/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_152/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_153/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_29:0' shape=(?, 1) dtype=float32>]
(1161619, 12) (23707, 12) (1161619,) (23707,)
4.332856986671189
25.62327837268502
19.241469879538815
4.3357236259332685
25.797232884801957
19.36866748217826
Train on 1161619 samples, validate on 23707 samples
Epoch 1/5
1161619/1161619 [==============================] - 18s 15us/step - loss: 1.0953 - val_loss: 0.3347
Epoch 2/5
1161619/1161619 [=============

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.0min remaining:    0.0s


[<tf.Tensor 'reshape_154/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_155/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_156/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_157/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_158/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_159/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_160/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_161/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_162/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_31:0' shape=(?, 1) dtype=float32>]
(1161619, 12) (23707, 12) (1161619,) (23707,)
4.332971482043596
25.60273635331378
19.215828942191887
4.327118572573501
25.920993799299787
19.504070527692242
Train on 1161619 samples, validate on 23707 samples
Epoch 1/5
1161619/1161619 [==============================] - 18s 16us/step - loss: 1.0946 - val_loss: 0.2730
Epoch 2/5
1161619/1161619 [=============

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 10.5min remaining:    0.0s


[<tf.Tensor 'reshape_163/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_164/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_165/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_166/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_167/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_168/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_169/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_170/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_171/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_33:0' shape=(?, 1) dtype=float32>]
(1161620, 12) (23707, 12) (1161620,) (23707,)
4.334197069609683
25.619374666414146
19.225807923417296
4.338802885223774
25.7503269076644
19.368751845446493
Train on 1161620 samples, validate on 23707 samples
Epoch 1/5
1161620/1161620 [==============================] - 19s 16us/step - loss: 1.1060 - val_loss: 0.2711
Epoch 2/5
1161620/1161620 [==============

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 14.0min remaining:    0.0s


[<tf.Tensor 'reshape_172/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_173/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_174/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_175/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_176/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_177/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_178/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_179/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_180/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_35:0' shape=(?, 1) dtype=float32>]
(1161620, 12) (23707, 12) (1161620,) (23707,)
4.333615123706548
25.628269141371533
19.234100652536974
4.329269835913443
25.640654658961488
19.260935588644703
Train on 1161620 samples, validate on 23707 samples
Epoch 1/5
1161620/1161620 [==============================] - 19s 17us/step - loss: 1.1046 - val_loss: 0.2638
Epoch 2/5
1161620/1161620 [============

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 17.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 17.6min finished


In [24]:
nnet2 = EM_NNRegressor(embed_cols=['brand_name','category_name','item_condition_id', 'cat1', 'cat2', 'cat3'], 
                  embed_dims=[(6000, 30),(1500, 25), (5,4), (15,4), (120, 10), (900, 20)],
                  text_embed_cols=['name', 'item_description', 'item_desc2gram'],
                  text_embed_dims=[(20000, 30), (50000, 30), (20000, 30)],
                  text_embed_seq_lens =[7, 70, 30],
                  text_embed_tokenizers = [tok_name, tok_desc, tok_desc2],
                  dense_cols=['shipping'],
                  epochs=4,
                  batchsize=2048 ,
                  num_layers = 1,
                  layer_dropouts=[0.2],
                  layer_dims=[100],
                  seed=2,
                  val_size=0.02
                 )
oof_preds2 = cross_val_predict(nnet1, X, y, verbose=10, cv=cvlist)
score = rmse(y, oof_preds2)
print(score)

[<tf.Tensor 'reshape_181/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_182/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_183/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_184/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_185/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_186/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_187/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_188/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_189/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_37:0' shape=(?, 1) dtype=float32>]
(1161619, 12) (23707, 12) (1161619,) (23707,)
4.334399661162567
25.62508447261968
19.236752325848666
4.335765807567385
25.815370987472054
19.345889399755347
Train on 1161619 samples, validate on 23707 samples
Epoch 1/5
1161619/1161619 [==============================] - 20s 17us/step - loss: 1.0900 - val_loss: 0.2981
Epoch 2/5
1161619/1161619 [=============

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s


[<tf.Tensor 'reshape_190/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_191/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_192/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_193/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_194/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_195/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_196/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_197/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_198/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_39:0' shape=(?, 1) dtype=float32>]
(1161619, 12) (23707, 12) (1161619,) (23707,)
4.332856986671189
25.62327837268502
19.241469879538815
4.3357236259332685
25.797232884801957
19.36866748217826
Train on 1161619 samples, validate on 23707 samples
Epoch 1/5
1161619/1161619 [==============================] - 20s 18us/step - loss: 1.0912 - val_loss: 0.2784
Epoch 2/5
1161619/1161619 [=============

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.1min remaining:    0.0s


[<tf.Tensor 'reshape_199/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_200/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_201/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_202/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_203/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_204/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_205/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_206/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_207/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_41:0' shape=(?, 1) dtype=float32>]
(1161619, 12) (23707, 12) (1161619,) (23707,)
4.332971482043596
25.60273635331378
19.215828942191887
4.327118572573501
25.920993799299787
19.504070527692242
Train on 1161619 samples, validate on 23707 samples
Epoch 1/5
1161619/1161619 [==============================] - 21s 18us/step - loss: 1.1067 - val_loss: 0.2973
Epoch 2/5
1161619/1161619 [=============

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 10.7min remaining:    0.0s


[<tf.Tensor 'reshape_208/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_209/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_210/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_211/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_212/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_213/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_214/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_215/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_216/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_43:0' shape=(?, 1) dtype=float32>]
(1161620, 12) (23707, 12) (1161620,) (23707,)
4.334197069609683
25.619374666414146
19.225807923417296
4.338802885223774
25.7503269076644
19.368751845446493
Train on 1161620 samples, validate on 23707 samples
Epoch 1/5
1161620/1161620 [==============================] - 21s 18us/step - loss: 1.1054 - val_loss: 0.2840
Epoch 2/5
1161620/1161620 [==============

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 14.4min remaining:    0.0s


[<tf.Tensor 'reshape_217/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_218/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_219/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_220/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_221/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_222/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_223/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_224/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_225/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_45:0' shape=(?, 1) dtype=float32>]
(1161620, 12) (23707, 12) (1161620,) (23707,)
4.333615123706548
25.628269141371533
19.234100652536974
4.329269835913443
25.640654658961488
19.260935588644703
Train on 1161620 samples, validate on 23707 samples
Epoch 1/5
1161620/1161620 [==============================] - 22s 19us/step - loss: 1.0881 - val_loss: 0.2575
Epoch 2/5
1161620/1161620 [============

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 18.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 18.0min finished


In [25]:
nnet3 = EM_NNRegressor(embed_cols=['brand_name','category_name','item_condition_id', 'cat1', 'cat2', 'cat3'], 
                  embed_dims=[(6000, 30),(1500, 20), (5,4), (15,4), (120, 10), (900, 20)],
                  text_embed_cols=['name', 'item_description', 'item_desc2gram'],
                  text_embed_dims=[(20000, 50), (50000, 50), (20000, 50)],
                  text_embed_seq_lens =[7, 60, 30],
                  text_embed_tokenizers = [tok_name, tok_desc, tok_desc2],
                  dense_cols=['shipping'],
                  epochs=4,
                  batchsize=2048 ,
                  num_layers = 1,
                  layer_dropouts=[0.2],
                  layer_dims=[200],
                  seed=3,
                  val_size=0.02,
                 )
oof_preds3 = cross_val_predict(nnet1, X, y, verbose=10, cv=cvlist)
score = rmse(y, oof_preds3)
print(score)

[<tf.Tensor 'reshape_226/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_227/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_228/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_229/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_230/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_231/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_232/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_233/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_234/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_47:0' shape=(?, 1) dtype=float32>]
(1161619, 12) (23707, 12) (1161619,) (23707,)
4.334399661162567
25.62508447261968
19.236752325848666
4.335765807567385
25.815370987472054
19.345889399755347
Train on 1161619 samples, validate on 23707 samples
Epoch 1/5
1161619/1161619 [==============================] - 22s 19us/step - loss: 1.0991 - val_loss: 0.2761
Epoch 2/5
1161619/1161619 [=============

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.7min remaining:    0.0s


[<tf.Tensor 'reshape_235/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_236/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_237/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_238/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_239/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_240/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_241/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_242/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_243/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_49:0' shape=(?, 1) dtype=float32>]
(1161619, 12) (23707, 12) (1161619,) (23707,)
4.332856986671189
25.62327837268502
19.241469879538815
4.3357236259332685
25.797232884801957
19.36866748217826
Train on 1161619 samples, validate on 23707 samples
Epoch 1/5
1161619/1161619 [==============================] - 23s 20us/step - loss: 1.1032 - val_loss: 0.2955
Epoch 2/5
1161619/1161619 [=============

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  7.4min remaining:    0.0s


[<tf.Tensor 'reshape_244/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_245/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_246/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_247/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_248/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_249/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_250/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_251/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_252/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_51:0' shape=(?, 1) dtype=float32>]
(1161619, 12) (23707, 12) (1161619,) (23707,)
4.332971482043596
25.60273635331378
19.215828942191887
4.327118572573501
25.920993799299787
19.504070527692242
Train on 1161619 samples, validate on 23707 samples
Epoch 1/5
1161619/1161619 [==============================] - 23s 20us/step - loss: 1.1015 - val_loss: 0.2894
Epoch 2/5
1161619/1161619 [=============

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 11.1min remaining:    0.0s


[<tf.Tensor 'reshape_253/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_254/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_255/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_256/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_257/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_258/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_259/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_260/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_261/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_53:0' shape=(?, 1) dtype=float32>]
(1161620, 12) (23707, 12) (1161620,) (23707,)
4.334197069609683
25.619374666414146
19.225807923417296
4.338802885223774
25.7503269076644
19.368751845446493
Train on 1161620 samples, validate on 23707 samples
Epoch 1/5
1161620/1161620 [==============================] - 24s 21us/step - loss: 1.1041 - val_loss: 0.3512
Epoch 2/5
1161620/1161620 [==============

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 14.9min remaining:    0.0s


[<tf.Tensor 'reshape_262/Reshape:0' shape=(?, 40) dtype=float32>, <tf.Tensor 'reshape_263/Reshape:0' shape=(?, 30) dtype=float32>, <tf.Tensor 'reshape_264/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_265/Reshape:0' shape=(?, 4) dtype=float32>, <tf.Tensor 'reshape_266/Reshape:0' shape=(?, 10) dtype=float32>, <tf.Tensor 'reshape_267/Reshape:0' shape=(?, 20) dtype=float32>, <tf.Tensor 'reshape_268/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_269/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'reshape_270/Reshape:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'dense_cols_55:0' shape=(?, 1) dtype=float32>]
(1161620, 12) (23707, 12) (1161620,) (23707,)
4.333615123706548
25.628269141371533
19.234100652536974
4.329269835913443
25.640654658961488
19.260935588644703
Train on 1161620 samples, validate on 23707 samples
Epoch 1/5
1161620/1161620 [==============================] - 24s 21us/step - loss: 1.0994 - val_loss: 0.3419
Epoch 2/5
1161620/1161620 [============

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 18.7min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 18.7min finished


In [26]:
from scipy.stats import hmean, gmean

In [27]:
oof_preds = np.mean(np.hstack((oof_preds1, oof_preds2, oof_preds3)), axis=1)
print(oof_preds.shape)
rmse(y, oof_preds)


(1481658,)
0.72281456 8.776944


0.41751682030956144

In [ ]:
#With Nadam lr=0.015, decay=0.005 - mean of 3 models is 0.4185 (0.429, 0.429, 0.428)
#RMSProp lr=0.003, decay=0.005 - mean 0.420 (0.425, 0.426, 0.424)
#RMSProp lr=0.01, decay=0.005 - mean 0.425 (0.434, 0.431, 0.430)
#RMSProp lr=0.004, decay=0.004 - mean 0.421 (0.428, 0.429, 0.427) 
#Adam lr=0.003, decay=0.003 - mean -- (0.4)
#With Nadam lr=0.012, decay=0.01 - mean of 3 models is 0.4175 (0.427, 0.428, 0.429)

In [ ]:
nnet1.fit(train_data, np.log1p(train_data.price) )
print("Predicting on test data")
test_preds1 = nnet1.predict(test_data)

nnet2.fit(train_data, np.log1p(train_data.price) )
print("Predicting on test data")
test_preds2 = nnet2.predict(test_data)

nnet3.fit(train_data, np.log1p(train_data.price) )
print("Predicting on test data")
test_preds3 = nnet3.predict(test_data)

test_preds = (1/3)*(test_preds1 + test_preds2 + test_preds3)
print("Write out submission")
submission: pd.DataFrame = test_data[['test_id']]
submission['price'] = np.expm1(test_preds)
submission.price = submission.price.clip(3, 2000)
submission.to_csv("embedding_nn_v2.csv", index=False)